# Предсказание оттока клиентов

Данный **учебный** проект - пример построения модели бинарной классификации, прдескакзывающей факт ухода клиента из банка в ближайшее время. 

In [1]:
import pandas as pd
import warnings 

warnings.filterwarnings('ignore')






In [2]:
data = pd.read_csv('/Users/alexey_zalesov/Desktop/ya_prakrikum/ds/datasets/Churn.csv')

display(data.head())
display(data.info())
display(data.describe())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


## Описание данных


**Признаки**

RowNumber — индекс строки в данных  
CustomerId — уникальный идентификатор клиента  
Surname — фамилия  
CreditScore — кредитный рейтинг  
Geography — страна проживания  
Gender — пол  
Age — возраст  
Tenure — сколько лет человек является клиентом банка  
Balance — баланс на счёте  
NumOfProducts — количество продуктов банка, используемых клиентом  
HasCrCard — наличие кредитной карты  
IsActiveMember — активность клиента  
EstimatedSalary — предполагаемая зарплата  
  
**Целевой признак**  

Exited — факт ухода клиента  


## Предварительный анализ

Есть пропуски в столбце "Tenure", столбец "RowNumber" дублирует индекс, также нужно изменить типы данных в некоторых столбцах. Также нужно привести все названия столбцов к строчному виду, проверить дубликаты и выбросы. 

 


In [3]:
data.columns = data.columns.str.lower()
data['surname'] = data['surname'].str.lower()
data['geography'] = data['geography'].str.lower()
data['gender'] = data['gender'].str.lower()

In [4]:
data = data.drop(columns=['rownumber'], axis=1)
display(data.head(10))


,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,15634602,hargrave,619,france,female,42,2.0,0.00,1,1,1,101348.88,1
1,15647311,hill,608,spain,female,41,1.0,83807.86,1,0,1,112542.58,0
2,15619304,onio,502,france,female,42,8.0,159660.80,3,1,0,113931.57,1
3,15701354,boni,699,france,female,39,1.0,0.00,2,0,0,93826.63,0
4,15737888,mitchell,850,spain,female,43,2.0,125510.82,1,1,1,79084.10,0
5,15574012,chu,645,spain,male,44,8.0,113755.78,2,1,0,149756.71,1
6,15592531,bartlett,822,france,male,50,7.0,0.00,2,1,1,10062.80,0
7,15656148,obinna,376,germany,female,29,4.0,115046.74,4,1,0,119346.88,1
8,15792365,he,501,france,male,44,4.0,142051.07,2,0,1,74940.50,0
9,15592389,h?,684,france,male,27,2.0,134603.88,1,1,1,71725.73,0


In [5]:
#проверяем на выбросы

display(data['customerid'].value_counts(ascending=False).head())
display(data['surname'].value_counts(ascending=False).head(20))
display(data['gender'].value_counts())
display(data['tenure'].value_counts())
display(data['geography'].value_counts())

display(data.duplicated().sum())

15812607    1
15741078    1
15635776    1
15740223    1
15738174    1
Name: customerid, dtype: int64

smith       32
scott       29
martin      29
walker      28
brown       26
yeh         25
shih        25
genovese    25
wright      24
maclean     24
white       23
fanucci     23
ma          23
wilson      23
johnson     22
wang        22
lu          22
moore       22
chu         22
mai         21
Name: surname, dtype: int64

male      5457
female    4543
Name: gender, dtype: int64

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

france     5014
germany    2509
spain      2477
Name: geography, dtype: int64

0

In [6]:
#разбираемся с пропусками в столбце "tenure"

print(data['tenure'].median())

data['tenure'] = data['tenure'].fillna(data['tenure'].median())

5.0


Решил заменить на медиану, т.к. пропущенных данных около 10% от общего числа объектов, а медиана меньше всего влияет на распределение. Медиана = 5, при этом разброс значений от 1 до 10, поэтому замена на медиану показалось самым подходящим вариантом. 

In [7]:
data['tenure'] = data['tenure'].astype('int')

data.info()
#проверили, все сработало

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customerid       10000 non-null  int64  
 1   surname          10000 non-null  object 
 2   creditscore      10000 non-null  int64  
 3   geography        10000 non-null  object 
 4   gender           10000 non-null  object 
 5   age              10000 non-null  int64  
 6   tenure           10000 non-null  int64  
 7   balance          10000 non-null  float64
 8   numofproducts    10000 non-null  int64  
 9   hascrcard        10000 non-null  int64  
 10  isactivemember   10000 non-null  int64  
 11  estimatedsalary  10000 non-null  float64
 12  exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


### Также удалим столбец 'surname'
Так как для идентификации можно использовать 'customer_id', а значения столбца 'surname', скорее всего, помешают модели обучится, так как фамилия человека не влияет на его решение уйти из банка

In [8]:
data = data.drop(columns=['surname'], axis=1)

### Краткий вывод
Удалили лишний столбец-индекс, изменили заглавные буквы в названиях столбцов и признаков на строчные, заменили пропуски в столбце 'tenure' на медиану этого столбца, изменили тип данных столбца на целочисленный. 


### Изменение категориальных признаков  

Требуется предсказать факт ухода клиентов в ближайшее время (1- клиент ушел , 0 - клиент остался). Значит, перед нами задача классификации. Также в столбцах 'geography','gender' и 'hascrcard' содержатся категориальные признаки, которые следует изменить на количественные для лучшего обучения модели. 



In [9]:
data_ohe = pd.get_dummies(data=data, columns=['geography','gender','hascrcard'], drop_first=True)

data_ohe.head()

,customerid,creditscore,age,tenure,balance,numofproducts,isactivemember,estimatedsalary,exited,geography_germany,geography_spain,gender_male,hascrcard_1
0,15634602,619,42,2,0.00,1,1,101348.88,1,0,0,0,1
1,15647311,608,41,1,83807.86,1,1,112542.58,0,0,1,0,0
2,15619304,502,42,8,159660.80,3,0,113931.57,1,0,0,0,1
3,15701354,699,39,1,0.00,2,0,93826.63,0,0,0,0,0
4,15737888,850,43,2,125510.82,1,1,79084.10,0,0,1,0,1


### Разделяем данные на обучающую, валидационную и тестовую выборки



In [10]:
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(data_ohe)
#data_ohe_scaled = scaler.transform(data_ohe)


features = data_ohe.drop(columns=['exited'], axis=1)
target = data_ohe['exited']

features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state = 12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.4, random_state=12345)



### Посмотрим на соотношение классов

In [11]:
class_1_count = data_ohe.query('exited == 1')['age'].count()
class_0_count = data_ohe.query('exited == 0')['age'].count()

partition_class_1 = class_1_count/len(target)
partition_class_0 = 1 - partition_class_1

print('Доля класса "1"', partition_class_1)
print('Доля класса "0"', partition_class_0)

Доля класса "1" 0.2037
Доля класса "0" 0.7963


Видно, что доля класса "1" кратно меньше доли класса "0".

### Обучаем модель без учета дисбаланса классов

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



model_imbalanced = DecisionTreeClassifier(random_state=12345)

model_imbalanced.fit(features_train, target_train)

predicted_valid = model_imbalanced.predict(features_valid)

f1_score_imbalanced = f1_score(target_valid, predicted_valid)
precision_score_imbalanced = precision_score(target_valid, predicted_valid)
recall_score_imbalanced = recall_score(target_valid, predicted_valid)

predicted_train = model_imbalanced.predict(features_train)

print('На валидационной выборке:')
print('F1-мера обученной модели с несбалансированными классами', f1_score_imbalanced)
print('Точность обученной модели с несбалансированными классами', precision_score_imbalanced)
print('Полнота обученной модели с несбалансированными классами', recall_score_imbalanced)
print()
#print('На обучающей выборке:')
#print('F1-мера обученной модели с несбалансированными классами', f1_score(target_train, predicted_train))
#print('Точность обученной модели с несбалансированными классами', precision_score(target_train, predicted_train))
#print('Полнота обученной модели с несбалансированными классами', recall_score(target_train, predicted_train))




На валидационной выборке:
F1-мера обученной модели с несбалансированными классами 0.49652432969215493
Точность обученной модели с несбалансированными классами 0.48638132295719844
Полнота обученной модели с несбалансированными классами 0.5070993914807302



### Устраним дисбаланс классов и подберем лучшую модель



In [13]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression


result={}

for threshold in np.arange(0.0,1.0,0.05):
    model_imbalanced_2 = LogisticRegression(random_state=12345, class_weight={0:threshold, 1:1-threshold})
    model_imbalanced_2.fit(features_train, target_train)
    probabilities_valid = model_imbalanced_2.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:,1]
    score = roc_auc_score(target_valid, probabilities_one_valid)
    new_element={threshold:score}
    result.update(new_element)
    print("Порог:", threshold, 'AUC-Score:', score,'\n')
    

    
print('Значение порога при лучшем результате', max(result, key=result.get))



Порог: 0.0 AUC-Score: 0.5 

Порог: 0.05 AUC-Score: 0.5626883341080315 

Порог: 0.1 AUC-Score: 0.562583031874667 

Порог: 0.15000000000000002 AUC-Score: 0.5624224193773127 

Порог: 0.2 AUC-Score: 0.5623820003382435 

Порог: 0.25 AUC-Score: 0.5623522178884031 

Порог: 0.30000000000000004 AUC-Score: 0.5720538509239472 

Порог: 0.35000000000000003 AUC-Score: 0.5622203241819665 

Порог: 0.4 AUC-Score: 0.5621575683055169 

Порог: 0.45 AUC-Score: 0.5621628866001314 

Порог: 0.5 AUC-Score: 0.5621256585378306 

Порог: 0.55 AUC-Score: 0.5620831121809157 

Порог: 0.6000000000000001 AUC-Score: 0.5620278019169261 

Порог: 0.65 AUC-Score: 0.5620448204596922 

Порог: 0.7000000000000001 AUC-Score: 0.5619714279940138 

Порог: 0.75 AUC-Score: 0.561960791404785 

Порог: 0.8 AUC-Score: 0.5618618711249577 

Порог: 0.8500000000000001 AUC-Score: 0.5617735874343589 

Порог: 0.9 AUC-Score: 0.5617459323023641 

Порог: 0.9500000000000001 AUC-Score: 0.5616087203013134 

Значение порога при лучшем результате 0.300

### Вывод
Следует установить значение порога в 0.3

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


model_tree = DecisionTreeClassifier(random_state=12345, class_weight={0:0.3, 1:0.7})
model_tree.fit(features_train, target_train)
predicted_tree_valid = model_tree.predict(features_valid)
print('F1-мера для решающего дерева', f1_score(target_valid, predicted_tree_valid))


model_forest = RandomForestClassifier(random_state=12345, class_weight={0:0.3, 1:0.7})
model_forest.fit(features_train, target_train)
predicted_forest_valid = model_forest.predict(features_valid)
print('F1-мера для случайного леса', f1_score(target_valid, predicted_forest_valid))


model_log_regr = LogisticRegression(random_state=12345, class_weight={0:0.3, 1:0.7})
model_log_regr.fit(features_train, target_train)
predicted_log_regr_valid = model_log_regr.predict(features_valid)
print('F1-мера для логистической регрессии', f1_score(target_valid, predicted_log_regr_valid))




F1-мера для решающего дерева 0.4778225806451613
F1-мера для случайного леса 0.5395973154362417
F1-мера для логистической регрессии 0.07653910149750416


### Случайный лес показал лучший результат, возьмем эту модель и подберем подходящие гиперпараметры. 

In [15]:
my_dict={}

for depth in range (3,40,1):
    final_model = RandomForestClassifier(random_state=12345, class_weight={0:0.3, 1:0.7}, max_depth=depth)
    final_model.fit(features_train, target_train)
    predicted_valid = final_model.predict(features_valid)
    score=f1_score(target_valid, predicted_valid)
    new_item = {depth:score}
    my_dict.update(new_item)
    print('Глубина', depth, 'F1-мера:', score,'\n')

    
print('Значение глубины при лучшем результате', max(my_dict, key=my_dict.get))    
    
    

Глубина 3 F1-мера: 0.5535499398315283 

Глубина 4 F1-мера: 0.5866364665911664 

Глубина 5 F1-мера: 0.5929304446978336 

Глубина 6 F1-мера: 0.592090395480226 

Глубина 7 F1-мера: 0.6002265005662514 

Глубина 8 F1-мера: 0.5952109464082099 

Глубина 9 F1-мера: 0.6123853211009174 

Глубина 10 F1-мера: 0.6020761245674741 

Глубина 11 F1-мера: 0.59375 

Глубина 12 F1-мера: 0.589242053789731 

Глубина 13 F1-мера: 0.5779467680608366 

Глубина 14 F1-мера: 0.5714285714285715 

Глубина 15 F1-мера: 0.5513654096228868 

Глубина 16 F1-мера: 0.5580182529335072 

Глубина 17 F1-мера: 0.5578947368421053 

Глубина 18 F1-мера: 0.5476190476190477 

Глубина 19 F1-мера: 0.5345744680851064 

Глубина 20 F1-мера: 0.5456953642384107 

Глубина 21 F1-мера: 0.5505319148936171 

Глубина 22 F1-мера: 0.5372340425531915 

Глубина 23 F1-мера: 0.5425531914893617 

Глубина 24 F1-мера: 0.5486018641810919 

Глубина 25 F1-мера: 0.5381526104417671 

Глубина 26 F1-мера: 0.5393858477970628 

Глубина 27 F1-мера: 0.53959731543624

In [16]:
my_dict={}

for estimator in range (3,100,1):
    final_model = RandomForestClassifier(random_state=12345, class_weight={0:0.3, 1:0.7},
                                         max_depth=9, n_estimators=estimator)
    final_model.fit(features_train, target_train)
    predicted_valid = final_model.predict(features_valid)
    score=f1_score(target_valid, predicted_valid)
    new_item = {estimator:score}
    my_dict.update(new_item)
    print('Количество листьев:', estimator, 'F1-мера:', score,'\n')

    
print('Количество листьев при лучшем результате', max(my_dict, key=my_dict.get))    
    
    

Количество листьев: 3 F1-мера: 0.5567451820128481 

Количество листьев: 4 F1-мера: 0.5470653377630122 

Количество листьев: 5 F1-мера: 0.5628476084538376 

Количество листьев: 6 F1-мера: 0.5682574916759157 

Количество листьев: 7 F1-мера: 0.5739910313901345 

Количество листьев: 8 F1-мера: 0.5825892857142858 

Количество листьев: 9 F1-мера: 0.5704697986577181 

Количество листьев: 10 F1-мера: 0.5745856353591162 

Количество листьев: 11 F1-мера: 0.568888888888889 

Количество листьев: 12 F1-мера: 0.5740112994350282 

Количество листьев: 13 F1-мера: 0.5662514156285391 

Количество листьев: 14 F1-мера: 0.576271186440678 

Количество листьев: 15 F1-мера: 0.5802047781569966 

Количество листьев: 16 F1-мера: 0.5785876993166288 

Количество листьев: 17 F1-мера: 0.5952109464082099 

Количество листьев: 18 F1-мера: 0.5940366972477064 

Количество листьев: 19 F1-мера: 0.5845799769850402 

Количество листьев: 20 F1-мера: 0.5842956120092379 

Количество листьев: 21 F1-мера: 0.5816917728852837 

Ко

### Обучим итоговую модель сразу на обучающей и валидационной выборках, проведем тестирование 

Обучаем итоговую модель с вручную заданным порогом:


In [17]:
from sklearn.utils import shuffle


features_train_valid , target_train_valid = shuffle(pd.concat([features_train, features_valid]),
                                                    pd.concat([target_train, target_valid]),
                                                    random_state=12345)

model = RandomForestClassifier(random_state=12345, class_weight={0:0.3, 1:0.7},
                                         max_depth=9, n_estimators=97)
model.fit(features_train_valid, target_train_valid)
predicted_test = model.predict(features_test)
score = f1_score(target_test, predicted_test)

probabilities_test = model.predict_proba(features_test)


print('F1-мера итоговой модели на тестовой выборке:', score)
print('AUC-score:', roc_auc_score(target_test, predicted_test))



F1-мера итоговой модели на тестовой выборке: 0.6176470588235294
AUC-score: 0.7415996474606147


Также посчитаем полноту и точность:

In [18]:
from sklearn.metrics import precision_score 


print('Точность итоговой модели на тестовой выборке:', precision_score(target_test, predicted_test))
print('Полнота итоговой модели на тестовой выборке:', recall_score(target_test, predicted_test))

Точность итоговой модели на тестовой выборке: 0.7159090909090909
Полнота итоговой модели на тестовой выборке: 0.5431034482758621


In [19]:
model_balanced_true = RandomForestClassifier(random_state=12345, class_weight='balanced',
                                         max_depth=9, n_estimators=97)


model_balanced_true.fit(features_train_valid, target_train_valid)
predicted_test = model_balanced_true.predict(features_test)
score = f1_score(target_test, predicted_test)

probabilities_test = model_balanced_true.predict_proba(features_test)


print('F1-мера модели c class_weight=''balanced'' на тестовой выборке:', score)
print('AUC-score:', roc_auc_score(target_test, predicted_test),'\n')

print('Точность модели c class_weight=''balanced'' на тестовой выборке:', precision_score(target_test, predicted_test))
print('Полнота модели c class_weight=''balanced'' на тестовой выборке:', recall_score(target_test, predicted_test))



F1-мера модели c class_weight=balanced на тестовой выборке: 0.6246575342465753
AUC-score: 0.7660846094524623 

Точность модели c class_weight=balanced на тестовой выборке: 0.5968586387434555
Полнота модели c class_weight=balanced на тестовой выборке: 0.6551724137931034


Обучим модель со сбалансированными классами методом upsample:

In [20]:
from sklearn.utils import shuffle

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)


#разделяем upsampled выборку
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features_upsampled, target_upsampled, test_size=0.4, random_state = 12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.4, random_state=12345)



#обучаем новую модель со сбалансированными классами методом upsample

model_upsample_balanced = RandomForestClassifier(random_state=12345,
                                         max_depth=9, n_estimators=97)

model_upsample_balanced.fit(features_train_valid, target_train_valid)
predicted_test = model_upsample_balanced.predict(features_test)
score = f1_score(target_test, predicted_test)

probabilities_test = model_upsample_balanced.predict_proba(features_test)


print('F1-мера модели c class_weight=''balanced'' на тестовой выборке:', score)
print('AUC-score:', roc_auc_score(target_test, predicted_test),'\n')

print('Точность модели c class_weight=''balanced'' на тестовой выборке:', precision_score(target_test, predicted_test))
print('Полнота модели c class_weight=''balanced'' на тестовой выборке:', recall_score(target_test, predicted_test))




F1-мера модели c class_weight=balanced на тестовой выборке: 0.6742160278745645
AUC-score: 0.7521498656758491 

Точность модели c class_weight=balanced на тестовой выборке: 0.9822335025380711
Полнота модели c class_weight=balanced на тестовой выборке: 0.513262599469496


### Вывод

Данные были очищены, поставлена задача классификации; данные были разбиты в соотношении 3:1:1, была построена и изучена модель без учета дисбаланса классов; затем дисбаланс был изучен и в ходе анализа AUC-ROC был подобран подходящий порог классов. Было проанализировано три модели: Логистическая регрессия, Решающее дерево и Случайный Лес. Последний был выбран так как показал изначально лучшее значение F1-меры, были подобраны соответствующие гиперпараметры, которые показали на тестовой выборке значение F1-меры в 0.617. а значение AUC-ROC: 0.741599.